# Mapping the Wikidata relations to CSKG relations

## Map relations and Prepare for CSKG

In [1]:
new_columns=['node1', 'relation', 'node2', 'node1;label', 'node2;label','relation;label', 'relation;dimension', 'source', 'sentence']

In [14]:
#tmp_dir='tmp/wikidata20200504'
tmp_dir='../tmp/wikidata20171227'#
#tmp_dir='tmp/wikidata20181210'#
wd_file='%s/wikidata_cs.tsv' % tmp_dir
mapped='%s/kgtk_wikidata.tsv' % tmp_dir
stats='%s/stats_mapped.tsv' % tmp_dir

In [15]:
import pandas as pd
import mapping

In [16]:
df1=pd.read_csv(wd_file, sep='\t')

In [17]:
len(df1)

104039

In [18]:
mapped_fw=set()
mapped_bw=set()
rows=[]
for i, row in df1.iterrows():
    node1=row[0]
    rel=row[1]
    node2=row[2]
    node1_label=row[3].strip()[1:-4]
    node2_label=row[4].strip()[1:-4]
    rel_label=row[5].strip()[1:-4]
    if rel in mapping.fw_mapping.keys() :
        cn_rel=mapping.fw_mapping[rel]
        my_row=[node1,cn_rel,node2,node1_label,node2_label,rel_label, '', 'WD', '']
    elif rel in mapping.bw_mapping.keys():
        cn_rel=mapping.bw_mapping[rel]
        my_row=[node2,cn_rel,node1,node2_label,node1_label,rel_label, '', 'WD', '']
    rows.append(my_row)


In [19]:
df2=pd.DataFrame(rows, columns=new_columns)

In [20]:
df2.drop_duplicates(inplace=True)

In [21]:
len(df2)

103992

In [22]:
df2['relation'].value_counts()

/r/IsA                74655
/r/PartOf              9186
/r/HasContext          5647
/r/HasProperty         3037
/r/RelatedTo           2034
/r/UsedFor             1928
/r/Antonym             1598
/r/DistinctFrom        1312
/r/Causes              1228
/r/HasPrerequisite     1000
/r/MadeOf               904
/r/Synonym              804
/r/DerivedFrom          410
/r/CreatedBy            197
/r/SimilarTo             52
Name: relation, dtype: int64

In [23]:
df2.to_csv(mapped, index=False, sep='\t', columns=new_columns)

In [24]:
%%bash -s "$mapped" "$stats"
kgtk graph_statistics --directed --degrees --pagerank --hits --log summary.txt -i $1 > $2

In [25]:
%%bash
cat summary.txt

loading the TSV graph now ...
graph loaded! It has 57885 nodes and 103992 edges

###Top relations:
/r/IsA	74655
/r/PartOf	9186
/r/HasContext	5647
/r/HasProperty	3037
/r/RelatedTo	2034
/r/UsedFor	1928
/r/Antonym	1598
/r/DistinctFrom	1312
/r/Causes	1228
/r/HasPrerequisite	1000

###Degrees:
in degree stats: mean=1.796528, std=0.123480, max=1
out degree stats: mean=1.796528, std=0.006967, max=1
total degree stats: mean=3.593055, std=0.124215, max=1

###PageRank
Max pageranks
710	Q336	0.010118
5046	P1918	0.012158
1958	Q7187	0.014300
284	Q151885	0.017690
9	Q2996394	0.013583

###HITS
HITS hubs
795	Q11173	0.000000
9	Q2996394	0.000000
5955	Q8054	0.000194
10239	Q20747295	0.000579
1958	Q7187	1.000000
HITS auth
35900	Q29663183	0.013885
10239	Q20747295	0.013888
55237	Q20969150	0.013893
42976	Q26738515	0.013893
10238	Q18256813	0.013893
